In [37]:
import time
from pywinauto.application import Application
from pywinauto.keyboard import send_keys

In [ ]:
app = Application(backend='uia').start(r"C:\\Program Files (x86)\Quantachrome Instruments\VersaWin\VersaWin.exe")
time.sleep(0.3)

In [ ]:
window = app.window(best_match='Quantachrome')
time.sleep(0.3)

In [ ]:
window.menu_select("0->2")

In [ ]:
file_dlg = app.window(best_match="Открыть", found_index=0)

In [ ]:
time.sleep(1)
file_dlg.child_window(best_match="Имя файла:Edit").type_keys(r"001-ASMS_St1_2011_06_16_01.QPS")

In [ ]:
file_dlg.child_window(best_match="Открыть4", control_type="Button").click()

In [ ]:
isotherm_window = app.window(best_match='Dialog')

In [ ]:
isotherm_window.click_input(button='right')
time.sleep(0.5)
send_keys('{DOWN}')
time.sleep(0.3)
send_keys('{DOWN}')
time.sleep(0.3)
send_keys('{ENTER}')
for i in range(13):
    time.sleep(0.3)
    send_keys('{DOWN}')
time.sleep(0.3)
send_keys('{ENTER}')
time.sleep(0.3)
send_keys('{ENTER}')
time.sleep(0.3)
######################

In [ ]:
confirmWin = app.window(best_match=u'Quantachrome® VersaWin™ Confirmation') 

In [ ]:
confirmWin.set_focus()
yesBtn = confirmWin[u'&Да']
            # Check the object name of the Yes button. You can use Swapy tool(It is deprecated but it works, else you can use inspect.exe)
yesBtn.click()

In [46]:
confirmWin = app.window(best_match=u'Quantachrome® VersaWin™ Confirmation') 

In [47]:
confirmWin.set_focus()
yesBtn = confirmWin[u'&Да']
            # Check the object name of the Yes button. You can use Swapy tool(It is deprecated but it works, else you can use inspect.exe)
yesBtn.click()

<uia_controls.ButtonWrapper - 'Да', Button, 7822299182997692518>

In [48]:
save_dlg = app.window(best_match="Name file", found_index=0)

In [49]:
save_dlg.child_window(best_match="Сохранить4", control_type="Button").click()

MatchError: Could not find 'Name file' in 'dict_keys(['Quantachrome® VersaWin™', 'Dialog', 'Quantachrome® VersaWin™Dialog'])'

In [ ]:
report_dlg = app.window(best_match="Dialog", found_index=0)

In [ ]:
report_dlg.child_window(best_match="Закрыть", control_type="Button").click()

In [ ]:
save_dlg.child_window(best_match="Сохранить4", control_type="Button").click()

In [ ]:
report_dlg = app.window(best_match="Dialog", found_index=0)

In [ ]:
report_dlg.child_window(best_match="Закрыть", control_type="Button").click()

In [12]:
from os import listdir
from os.path import isfile, join

QPS_path = r"C:\QCdata\Physisorb"
files = [f for f in listdir(QPS_path) if isfile(join(QPS_path, f))]
path_to_ready_files = r"C:\QCdata\Export"
redy_files = [f for f in listdir(path_to_ready_files) if isfile(join(path_to_ready_files, f))]

def get_unprocessed(files, redy_files):
    files_ = set([f.replace(".QPS", "").replace(".qps", "") for f in files])
    redy_files_ = set([f.replace(".txt", "").replace(" (Isotherm)", "") for f in redy_files])
    rest = files_ - redy_files_
    rest_files = [f + ".QPS" for f in rest]
    return redy_files

(38, 206)

In [7]:
import struct


def read_null_terminated_string(file):
    chars = []
    while True:
        char = file.read(1)
        if char == b'\0' or not char:  # Остановка на \0 или в конце файла
            break
        chars.append(char.decode('ascii', errors='ignore'))
    return ''.join(chars)

with open("test.QPS", "rb") as f:
    rows = []
    while True:
        try:
            # Считываем строку, заканчивающуюся \0
            string_data = read_null_terminated_string(f)
            
            # Считываем число после строки (4 байта для float)
            number_data = struct.unpack('f', f.read(4))[0]
            
            rows.append((string_data, number_data))
        except struct.error:  # Прекращаем при достижении конца файла
            break

# Вывод результатов
for row in rows:
    print(row)


KeyboardInterrupt: 

In [25]:
import struct

def extract_strings_and_floats(file_path, min_val=0.0, max_val=1.0):
    strings = []
    floats = []
    with open(file_path, "rb") as f:
        current_string = []
        
        while True:
            # Чтение 1 байта для проверки на ASCII-символ
            char = f.read(1)
            if not char:  # Конец файла
                break
            
            # Проверяем, является ли байт ASCII-символом (от 32 до 126)
            if 32 <= ord(char) < 127:
                current_string.append(char.decode('ascii'))
            else:
                # Если текущий фрагмент накопленных символов завершён, добавляем его в список строк
                if current_string:
                    strings.append(''.join(current_string))
                    print(''.join(current_string))
                    current_string = []
                
                # Возвращаемся на один байт назад, чтобы не пропустить символ при следующем чтении
                f.seek(-1, 1)
                
                # Чтение 4 байтов для попытки интерпретации как float32
                float_data = f.read(4)
                if len(float_data) == 4:  # Проверяем, что данных достаточно для float
                    try:
                        (number,) = struct.unpack("f", float_data)
                        # Проверяем, входит ли число в заданный диапазон
                        if min_val <= number <= max_val:
                            print(number)
                            floats.append(number)
                    except struct.error:
                        # Если распаковка невозможна, продолжаем
                        continue

    # Добавляем последнюю строку, если она накопилась
    if current_string:
        strings.append(''.join(current_string))
        
    return strings, floats

# Пример использования
strings, floats = extract_strings_and_floats("test.QPS")
# print("Извлеченные строки:", strings)
# print("Извлеченные числа:", floats)


QBINa
2.3512756875700687e-38
2.350988701644575e-38
1.5281426560689738e-36
3.576562096465517e-40
5.520891741685487e-40
Header
ileVersion
inVersion
ecWarn
drType
W
dminData
nalysisMode
naModeParam
naModeParam2
vMode
vCellSize
vHeEvacMins
vColdZoneVolume
vWarmZoneVolume
sedFastInit
ressToleranceAds
ressToleranceDes
quilibriumTimeAds
quilibriumTimeDes
imeOutAds
imeOutDes
hermalDelay
hermalComp
rigDataTherm
utgasTemp
utgasTime
ellId
ampleVol
oType
vacCrossOver
dsCrossSectionArea
dsMolecWeight
dsNonIdeality
anTempComp
anThermTransp
drVersion
ampleId
i 15
ampleDesc
arfenov:Si 15
peratorId
yupov
omments
3.761583267877846e-37
bientTemp
athTemp
ampleWeight
nstrumentType
uadraSorb SI MP
nstVerString
7.63032858230872e-06
.01
cVersion
unData
tationId
1.8627026587836326e-09
tartTime
0.000122137411381118
/29/2024 17:02:02
ndTime
0.000122137411381118
/30/2024 15:25:14
imeElapsed
serSetup
ovaFileName
492901.dat
ampleDens
ellVolume
ctualPo
eportedPo
dsName
itrogen
nalysisDate
0.00012210388376843184
/30/